In [81]:
dtype_dict = {
    'id': 'int64',
    'outbalance': 'Int64',
    'fc': 'float64',
    'ref': 'Int64',
    'regDate': 'str',
    'totalbonus': 'float64',
    'payout': 'float64',
    'userActivatedPromo': 'float64',
    'premium': 'float64',
    'ban': 'float64',
    'refCount': 'float64',
    'not': 'float64',
    'data': 'float64',
    'last_bonus_day': 'float64',
    'last_com_day': 'float64',
    'bonuscount': 'float64',
    'imgay': 'float64',
    'proverka': 'float64',
    'check': 'float64',
    'subs': 'float64',
    # Handle completedTasks and usedPromoCodes as object type initially
}

# Import the CSV with dtype specified
mongo_df = pd.read_csv('c90758_geckoshig_bot_2.users.csv', dtype=dtype_dict, low_memory=False)


In [94]:
import numpy as np
import pandas as pd
mongo_df = mongo_df.rename(columns={
    'id': 'telegram_id',
    'outbalance': 'balance',
    'ref': 'referred_by_id',
    'ban': 'blocked',
    'premium': 'is_premium',
    'regDate': 'created_at',
    # Map other columns if needed
})

# Convert data types to match PostgreSQL schema
mongo_df['telegram_id'] = mongo_df['telegram_id'].astype(int)
mongo_df['balance'] = mongo_df['balance'].fillna(0).astype(int)
mongo_df['bmeme_balance'] = 0  # Default value
mongo_df['bmeme_balance'] = mongo_df['bmeme_balance'].astype(int)
mongo_df['referred_by_id'] = mongo_df['referred_by_id'].astype('Int64')  
mongo_df['referred_by_id'] = mongo_df['referred_by_id'].where(pd.notna(mongo_df['referred_by_id']), None)

mongo_df['blocked'] = mongo_df['blocked'].fillna(0).astype(bool)  
mongo_df['is_premium'] = mongo_df['is_premium'].fillna(0).astype(bool)
mongo_df['created_at'] = pd.to_datetime(mongo_df['created_at'], errors='coerce', dayfirst=True)

# Handle specific date parsing issues if necessary
# Example for replacing specific values:
mongo_df['created_at'] = mongo_df['created_at'].replace(pd.to_datetime(1490195805433502912, unit='ns'), pd.NaT)

# Remove duplicates based on 'telegram_id'
mongo_df = mongo_df.drop_duplicates(subset='telegram_id', keep='first')

# Filter rows where 'telegram_id' > 10
mongo_df = mongo_df[mongo_df['telegram_id'] > 10]

# Handle missing or boolean columns
mongo_df['is_bot_start_completed'] = True  # Default value
mongo_df['is_admin'] = False  # Default value
mongo_df['language'] = 'EN'  # Default language
mongo_df['deleted_at'] = None  # Default None for deleted_at

# Convert NaN and NA to None
def convert_na_to_none(value):
    # Check if value is NaN or NA and return None if true
    if pd.isna(value):
        return None
    return value

# Apply conversion to all columns
mongo_df = mongo_df.map(convert_na_to_none)

# Select only the columns needed for PostgreSQL
postgres_df = mongo_df[[
    'telegram_id', 
    'balance', 
    'bmeme_balance',  # Ensure this column exists
    'referred_by_id', 
    'blocked', 
    'language', 
    'is_admin', 
    'is_premium', 
    'is_bot_start_completed', 
    'created_at', 
    'deleted_at'
]]
print(postgres_df.head())

   telegram_id  balance  bmeme_balance  referred_by_id  blocked language  \
2   5675315935    50001              0             NaN    False       EN   
3    572954023    52511              0             NaN    False       EN   
4   6843107863    53516              0             NaN    False       EN   
5   5774025780    50001              0             NaN    False       EN   
6   1629000161    50001              0             NaN    False       EN   

   is_admin  is_premium  is_bot_start_completed created_at deleted_at  
2     False       False                    True 2024-05-05       None  
3     False       False                    True 2024-05-05       None  
4     False        True                    True 2024-05-05       None  
5     False       False                    True 2024-05-05       None  
6     False       False                    True 2024-05-05       None  


In [89]:
import os
from enum import Enum
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session, scoped_session
import datetime
import uuid
from typing import Optional


from sqlalchemy import Column, DateTime, ForeignKey, BigInteger, Enum as SQLEnum, Numeric, Text, func
from sqlalchemy.dialects.postgresql import UUID as PG_UUID, JSONB
from sqlalchemy.orm import Mapped, mapped_column, relationship

from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.orm import sessionmaker

data_to_insert = postgres_df.to_dict(orient='records')

# Define the metadata and table
metadata = MetaData()
Base = sqlalchemy.orm.declarative_base()


class Lang(Enum):
    RU = "russian"
    EN = "english"
    TR = "turkish"
    DE = "german"


class User(Base):
    __tablename__ = 'users'

    telegram_id: Mapped[int] = mapped_column(type_=BigInteger, primary_key=True)
    balance: Mapped[int] = mapped_column(type_=Numeric, default=0)
    bmeme_balance: Mapped[int] = mapped_column(type_=Numeric, default=0)
    referred_by_id: Mapped[Optional[int]] = mapped_column(ForeignKey('users.telegram_id'), type_=BigInteger)
    blocked: Mapped[bool] = mapped_column(default=False)
    language: Mapped[Lang] = mapped_column(SQLEnum(Lang), default=Lang.EN)
    is_admin: Mapped[bool] = mapped_column(default=False, nullable=False)
    is_premium: Mapped[bool] = mapped_column(default=False, nullable=False)
    is_bot_start_completed: Mapped[bool] = mapped_column(default=False)
    created_at = mapped_column("created_at", DateTime, index=True)
    deleted_at = Column(DateTime, default=None)

# Create a new session
Session = sessionmaker(bind=engine)
session = Session()

# Perform bulk insert with conflict handling

    # Use bulk_insert_mappings for efficient bulk insert
session.bulk_insert_mappings(User, data_to_insert)
session.commit()


OperationalError: (psycopg2.OperationalError) SSL SYSCALL error: EOF detected

[SQL: INSERT INTO users (telegram_id, balance, bmeme_balance, referred_by_id, blocked, language, is_admin, is_premium, is_bot_start_completed, created_at) VALUES (%(telegram_id__0)s, %(balance__0)s, %(bmeme_balance__0)s, %(referred_by_id__0)s, %(blocked__0 ... 220704 characters truncated ... 999)s, %(is_admin__999)s, %(is_premium__999)s, %(is_bot_start_completed__999)s, %(created_at__999)s)]
[parameters: {'language__0': 'EN', 'telegram_id__0': 5675315935, 'is_bot_start_completed__0': True, 'blocked__0': False, 'is_premium__0': False, 'created_at__0': Timestamp('2024-05-05 00:00:00'), 'bmeme_balance__0': 0, 'referred_by_id__0': nan, 'is_admin__0': False, 'balance__0': 50001, 'language__1': 'EN', 'telegram_id__1': 572954023, 'is_bot_start_completed__1': True, 'blocked__1': False, 'is_premium__1': False, 'created_at__1': Timestamp('2024-05-05 00:00:00'), 'bmeme_balance__1': 0, 'referred_by_id__1': nan, 'is_admi